# PepperOpenDayDemo

## Environment Setup

In [1]:
# built-in imports
# !pip install requests
# !pip install speechrecognition==3.8.1
# !pip install paramiko
# !pip install pandas
# !pip install beautifulsoup4
# !pip install pocketsphinx
# !pip install scipy

In [1]:
from traceback import format_exc
from time      import sleep

from idle import idling

from interactiveControls import EyesController
from interactiveControls import track_head
from interactiveControls import return_to_default_pos

from humanInteraction import detect
from humanInteraction import listen
from humanInteraction import speak
from humanInteraction import faceTracker
from humanInteraction import processQuery
from humanInteraction import stopListening

from chatflow import convInit, convFlow

from behaviorManager import launchAndStopBehavior, service

/usr/local/lib/python2.7/site-packages/paramiko/transport.py:33: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography.hazmat.backends import default_backend


## Constants

In [2]:
### Networking Constants ###
GPT_HOST = "10.104.23.208"
PEPPER_HOST = "10.104.23.185"
GPT_PORT = 8891
PEPPER_PORT = 9559

### Audio Processing Constants ###
PEPPER_RECORDINGS_PATH = "/home/nao/microphones/recording.wav"
SERVER_RECORDINGS_PATH = "recordings/recording.wav"

### Display Generation Constants ###
FILE_NAME_TEMP = "display.html"
PEPPER_HTML_PATH = "/home/nao/.local/share/PackageManager/apps/robot-page/html/"  # page.html"
PEPPER_PAGE_LANDING = PEPPER_HTML_PATH + "page.html" # '/home/nao/.local/share/PackageManager/apps/robot-page/html/page.html'
PEPPER_QR_LANDING = PEPPER_HTML_PATH + "webfiles/qr.png"  # switched slash direction
PEPPER_IMG_LANDING = PEPPER_HTML_PATH + "webfiles/img.png"  # switched slash direction
TEXT_BY_ID_PATH = "textbyID.csv"

# Choregraph Behavior

In [3]:
### Guitar Behaviour ###
launchAndStopBehavior(service, '.lastUploadedChoregrapheBehavior/behavior_1')

## MAIN

In [5]:
global idle
global responsesPipeline
global eyes

idle = idling()
eyes = EyesController()
eyes.startEyes("neutral")

try:
    track_head()

    while True:
        # Empty sentences list
        responsesPipeline = []
        
        # Idle behaviors to keep peppers temperature low
        idle.start_idle_behavior()

        # trigger smart listen
        detect(idle)
        faceTracker()
        # confusionCount = 0 # confusion level: number of times Pepper repeats herself and requests clarification
        state = {"topic": "fresh", "confusion": 0, "repeat": True, "confident": False}


        while state["repeat"]:
            query = listen(eyes)
        
            state = processQuery(query, responsesPipeline, eyes, state)
            if state["confusion"] > 3:
                state["repeat"] = False
                raise Exception("tooHot")
            
            
            
            
        # if priorState == "error":
        #     confusionCount += 1



        # # confusion level: number of times Pepper repeats herself and requests clarification
        # confusionCount = 0

        # while confusionCount < 3: # while under max repeat threshold
        #     query = listen()
        #     repeat = convInit(query, responsesPipeline, eyes)

        #     # pred_class = classifyQuery(query)

        #     # if query[0] == "%":
        #         # speak(tieredConfusionPrompts[confusionCount])
        #         # confusionCount += 1
        #     # repeat = convFlow(query, pred_class, thresh, responsesPipeline, eyes, rCount)



        #     if query == '%low_volume_error%':

        #         speak(tieredConfusionPrompts[confusionCount])
        #         confusionCount += 1
        #     else:
        #         pred_class, queryIsAudible = classifyQuery(query)

        # rCount = 0 # repeat counter: how many times will she repeat herself
        # cCounter = 0 # Course Query: specific course
        # while rCount < 3: # while under max repeat threshold
        #     query = listen()
        #     if query == '%low_volume_error%':
        #         speak("Sorry, I didn't quite catch that. Could you please repeat that?")
        #         rCount += 1
        #     else:
        #         pred_class, thresh = classifyQuery(query)

        #         if not thresh: # 
        #             rCount += 1
        #         if pred_class == 'Courses':
        #             cCounter += 1
        #         if cCounter == 2:
        #             pred_class = 'SpecificCourse'

        #         repeat = convFlow(query, pred_class, thresh, responsesPipeline, eyes, rCount)

        #         if repeat:
        #             rCount += 1
        
        # When the user has tried 3 times, return to default position
        sleep(10)
        return_to_default_pos()
        continue
except Exception as e:
    print(e)
    print(format_exc())
finally:
    stopListening()
    return_to_default_pos()
    idle.start_idle_behavior()
    print('Goodbye')

copyfile successful
Sent to Pepper
0.611710429192
Detect complete
copyfile successful
Sent to Pepper
Recording is starting...
copyfile successful
Sent to Pepper
Timer is reached
Recording is stopping...
Sent from Pepper
[  -5  -15  -25 ... -120 -555 -265]
<type 'numpy.ndarray'>
Saved noise reduced audio file to recordings/rn_recording.wav
('Listen complete, This is what I heard: ', u'this is like a chi-square you should hopefully it the casual')
copyfile successful
Sent to Pepper
Recording is starting...
copyfile successful
Sent to Pepper
Timer is reached
Recording is stopping...
Sent from Pepper
[  0   0   0 ... -75   0 455]
<type 'numpy.ndarray'>
Saved noise reduced audio file to recordings/rn_recording.wav
('Listen complete, This is what I heard: ', u'no I was just talking about some gibberish')
copyfile successful
Sent to Pepper
Recording is starting...
copyfile successful
Sent to Pepper
Timer is reached
Recording is stopping...
Sent from Pepper
[ -5  -5  -5 ... 200 325 685]
<type 

displayGeneration.py:123: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 123 of the file displayGeneration.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  target.replace_with(BeautifulSoup(subText))


Sent to Pepper
Sent to Pepper
copyfile successful
Sent to Pepper
Recording is starting...
copyfile successful
Sent to Pepper
Timer is reached
Recording is stopping...
Sent from Pepper
[   10    15     5 ... -1200 -1250 -1175]
<type 'numpy.ndarray'>
Saved noise reduced audio file to recordings/rn_recording.wav
('Listen complete, This is what I heard: ', u'no I want to know what the weather is like today how are you feeling')
copyfile successful
Sent to Pepper
copyfile successful
Sent to Pepper
0.778601884842
Detect complete
copyfile successful
Sent to Pepper
Recording is starting...
copyfile successful
Sent to Pepper
Timer is reached
Recording is stopping...
Sent from Pepper
[    5     5     5 ... -2065 -2555 -2730]
<type 'numpy.ndarray'>
Saved noise reduced audio file to recordings/rn_recording.wav
('Listen complete, This is what I heard: ', u'I am looking for accommodation information')
copyfile successful
text match found
text match found
Sent to Pepper
Sent to Pepper
Sent to Pepper


KeyboardInterrupt: 